In [2]:
#importo las librerias necesarias - 551 el mejor modelo
#Incluyendo Grid

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.datasets import make_regression
from lightgbm import LGBMRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

In [3]:
#cargo el dataset de training y el de test

df_training = pd.read_csv('./../data/training_one_hot_no_cities_operations_result_data.csv')
df_test = pd.read_csv('./../data/diamonds_test_one_hot_no_cities_operations_result_data.csv')
df_training

,price,carat,depth,table,x,y,z,cut/weight_Fair,cut/weight_Good,cut/weight_Ideal,...,color/weight_I,color/weight_J,clarity/weight_I1,clarity/weight_IF,clarity/weight_SI1,clarity/weight_SI2,clarity/weight_VS1,clarity/weight_VS2,clarity/weight_VVS1,clarity/weight_VVS2
0,4268.00,1.21,62.4,58.0,6.83,6.79,4.25,0.0,0.0,0.000000,...,0.000000,0.826446,0.0,0.0,0.000000,0.000000,0.000000,0.826446,0.0,0.0
1,3513.00,1.02,61.6,58.0,6.40,6.35,3.93,0.0,0.0,0.000000,...,0.000000,0.980392,0.0,0.0,0.000000,0.000000,0.000000,0.980392,0.0,0.0
2,1792.00,0.77,62.3,58.0,5.86,5.80,3.63,0.0,0.0,0.000000,...,0.000000,1.298701,0.0,0.0,0.000000,0.000000,0.000000,1.298701,0.0,0.0
3,7553.00,1.51,59.6,60.0,7.58,7.48,4.49,0.0,0.0,0.000000,...,0.000000,0.662252,0.0,0.0,0.000000,0.000000,0.000000,0.662252,0.0,0.0
4,1176.00,0.57,60.2,62.0,5.40,5.33,3.23,0.0,0.0,0.000000,...,0.000000,1.754386,0.0,0.0,0.000000,0.000000,0.000000,1.754386,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53935,1976.56,0.57,61.9,56.0,5.35,5.32,3.30,0.0,0.0,1.754386,...,0.000000,0.000000,0.0,0.0,1.754386,0.000000,0.000000,0.000000,0.0,0.0
53936,2331.41,0.71,62.2,55.0,5.71,5.73,3.56,0.0,0.0,1.408451,...,1.408451,0.000000,0.0,0.0,0.000000,0.000000,0.000000,1.408451,0.0,0.0
53937,2910.38,0.70,61.6,55.0,5.75,5.71,3.53,0.0,0.0,1.428571,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,1.428571,0.000000,0.0,0.0
53938,2113.32,0.70,58.8,57.0,5.85,5.89,3.45,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,1.428571,0.000000,0.000000,0.0,0.0


In [4]:
# Features + target

X = df_training[['carat',
                    'depth',
                    'table',
                    'x',
                    'y',
                    'z',
                    'cut/weight_Fair',
                    'cut/weight_Good',
                    'cut/weight_Ideal',
                    'cut/weight_Premium',
                    'cut/weight_Very Good',
                    'color/weight_D',
                    'color/weight_E',
                    'color/weight_F',
                    'color/weight_G',
                    'color/weight_H',
                    'color/weight_I',
                    'color/weight_J',
                    'clarity/weight_I1',
                    'clarity/weight_IF',
                    'clarity/weight_SI1',
                    'clarity/weight_SI2',
                    'clarity/weight_VS1',
                    'clarity/weight_VS2',
                    'clarity/weight_VVS1',
                    'clarity/weight_VVS2']]
y = pd.get_dummies(df_training[['price']], drop_first=True)
print(X.shape,y.shape)

(53940, 26) (53940, 1)


In [5]:
# Train + test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Scaling - StandardScaler
scaler = RobustScaler()
scaled_data = scaler.fit_transform(X_train)
scaled_data_test = scaler.transform(X_test)
scaled_data_test

array([[-0.3125    ,  0.46666667,  0.66666667, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.5625    ,  0.33333333,  0.33333333, ...,  0.        ,
         0.        ,  0.        ],
       [ 2.34375   , -0.13333333,  0.        , ...,  0.45454545,
         0.        ,  0.        ],
       ...,
       [ 1.25      ,  0.53333333, -0.33333333, ...,  0.66666667,
         0.        ,  0.        ],
       [ 0.78125   ,  1.46666667,  1.33333333, ...,  0.        ,
         0.        ,  0.        ],
       [-0.46875   , -1.26666667,  1.        , ...,  0.        ,
         0.        ,  0.        ]])

In [7]:
regressor = LGBMRegressor()

In [8]:
hyperparameters = regressor.get_params()
model = regressor.fit(X_train, y_train.values.ravel())

In [9]:
y_pred = model.predict(X_test)

print('Model:', regressor, '\n')
print('Model hyperparameters:', hyperparameters, '\n')
#print('Ground truth target:', y_test, '\n')
print('Predicted target:', y_pred, '\n')

Model: LGBMRegressor() 

Model hyperparameters: {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0, 'importance_type': 'split', 'learning_rate': 0.1, 'max_depth': -1, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 100, 'n_jobs': -1, 'num_leaves': 31, 'objective': None, 'random_state': None, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'silent': 'warn', 'subsample': 1.0, 'subsample_for_bin': 200000, 'subsample_freq': 0} 

Predicted target: [ 1207.7899856  13499.89495843 14944.35441479 ... 13404.66191749
  6155.85162653   685.6656681 ] 



In [10]:
y_pred.shape

(10788,)

In [11]:
type(y_pred)

numpy.ndarray

In [12]:
r2_ = model.score(X_test, y_test)
r2_

0.9857851748457008

In [13]:
rmse = mean_squared_error(y_test, y_pred)**0.5
rmse

464.92615138673466

In [14]:
param_grid = {'n_estimators': [16, 32, 64, 128, 256, 512],
              'max_depth': [2, 4, 8, 16]}

In [15]:
grid_search = GridSearchCV(model,
                           param_grid,
                           cv=5,
                           verbose=3,
                           scoring='neg_root_mean_squared_error',
                           n_jobs=-1)

In [16]:
%%time

grid_search.fit(X,y)

print('\n')
print('Best hyperparameters: ', grid_search.best_params_, '\n')
print('Best score: ', -grid_search.best_score_, '\n')

Fitting 5 folds for each of 24 candidates, totalling 120 fits


Best hyperparameters:  {'max_depth': 16, 'n_estimators': 512} 

Best score:  461.60724666298864 

CPU times: total: 7.78 s
Wall time: 21.1 s


In [17]:
best_model = grid_search.best_estimator_

In [18]:
y_test = best_model.predict(df_test)

In [19]:
df_pred = pd.DataFrame(data=y_test)
df_pred

,0
0,2933.856018
1,5668.621878
2,9854.152673
3,3959.325766
4,1651.642189
...,...
13480,1685.053606
13481,2381.084541
13482,3129.954255
13483,2138.809466


In [20]:
df_pred = df_pred.reset_index()
df_pred

,index,0
0,0,2933.856018
1,1,5668.621878
2,2,9854.152673
3,3,3959.325766
4,4,1651.642189
...,...,...
13480,13480,1685.053606
13481,13481,2381.084541
13482,13482,3129.954255
13483,13483,2138.809466


In [21]:
df_rename = df_pred.rename(columns={"index": "id", "o": "price"})
df_rename

,id,0
0,0,2933.856018
1,1,5668.621878
2,2,9854.152673
3,3,3959.325766
4,4,1651.642189
...,...,...
13480,13480,1685.053606
13481,13481,2381.084541
13482,13482,3129.954255
13483,13483,2138.809466


In [22]:
final_prediction = df_rename.rename(columns={df_rename.columns[1]: "price" })
final_prediction

,id,price
0,0,2933.856018
1,1,5668.621878
2,2,9854.152673
3,3,3959.325766
4,4,1651.642189
...,...,...
13480,13480,1685.053606
13481,13481,2381.084541
13482,13482,3129.954255
13483,13483,2138.809466


In [23]:
export = final_prediction.to_csv('./../results/prediction_lgbm_grid.csv', index=False)